## Langchain vs Langgraph

LangChain and LangGraph are both libraries developed by LangChain Inc., but they serve different purposes within the ecosystem of building LLM (Large Language Model)-powered applications. Here's a clear breakdown of their differences:

🔷 LangChain
What it is: A framework for building applications with LLMs using chains, agents, tools, memory, and retrieval.

Primary abstraction: Chains (sequential steps of logic) and Agents (models that decide which tools to use at each step).

Use cases: Q&A bots, document retrieval systems, code assistants, customer support agents, etc.

Key components:

Chains: Sequence of LLM calls.

Agents: Dynamic decision-making using tools.

Memory: For stateful conversations.

Retrievers: For RAG (retrieval augmented generation).

🔶 LangGraph
What it is: A stateful computation framework built on top of LangChain, using graphs instead of chains.

Primary abstraction: State machines / Graphs — nodes are functions (often LLM calls), and edges represent conditional transitions.

Use cases: More complex workflows with loops, branches, parallelism, and state management.

Built on: NetworkX (Python graph library) + LangChain primitives.

Key feature: Can model multi-step, multi-agent, or recursive interactions more naturally than LangChain alone.

🧠 Analogy:
LangChain = Pipeline (linear, or dynamic with agents).

LangGraph = Flowchart/State Machine (more control over flow, branching, and re-entry).

### Use Case: Sentiment Analysis with Conditional Follow-Up
Input: User enters a sentence.

Step 1: Detect sentiment (positive/negative/neutral).

Step 2:

If positive: Reply with a cheerful response.

If negative: Ask a follow-up question.

If neutral: Thank the user.

## 1. Langchain approach

In [6]:
#from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#llm = ChatOpenAI()
llm = ChatOllama(model="llama2")  # or "mistral", "llama2", etc.

# Step 1: Sentiment detection
sentiment_prompt = PromptTemplate.from_template("You are a sentiment analysis tool. What is the sentiment of this sentence: '{text}'? (positive, negative, neutral)")
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt)

# Step 2: Define responses
def handle_sentiment(text):
    sentiment = sentiment_chain.run(text).strip().lower()

    if sentiment == "positive":
        return "That's great to hear! 😊"
    elif sentiment == "negative":
        return "I'm sorry to hear that. Want to talk about it?"
    else:
        return "Thanks for sharing that!"

# Run it
user_input = "I had a terrible day at work."
response = handle_sentiment(user_input)
print(response)


I'm sorry to hear that. Want to talk about it?


In [7]:
# Run it
user_input = "Its such a bright sunny day. I feel happy today."
response = handle_sentiment(user_input)
print(response)

Thanks for sharing that!


In [8]:
user_input = "It was a horrible iphone. It kept hanging. I found it very difficult to operate."
response = handle_sentiment(user_input)
print(response)

Thanks for sharing that!


In [9]:
user_input = "It was a great movie. I highly recommend. Lots of suspense"
response = handle_sentiment(user_input)
print(response)

Thanks for sharing that!


## 2. Langgraph approach

In [15]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
#from langchain.chat_models import ChatOpenAI

# Define schema
class SentimentState(TypedDict):
    input: str
    sentiment: str
    output: str

#llm = ChatOpenAI()

# Define nodes
def detect_sentiment(state: SentimentState) -> dict:
    text = state["input"]
    prompt = f"What is the sentiment of this sentence: '{text}'? Answer only one word from these 3 options : positive, negative, or neutral"
    result = llm.invoke(prompt)
    
    print ("result:", result)
    return {"sentiment": result.content.strip().lower()}

def handle_positive(state: SentimentState) -> dict:
    return {"output": "That's great to hear! 😊"}

def handle_negative(state: SentimentState) -> dict:
    return {"output": "I'm sorry to hear that. Want to talk about it?"}

def handle_neutral(state: SentimentState) -> dict:
    return {"output": "Thanks for sharing that!"}

# Build the graph with state schema
builder = StateGraph(SentimentState)

builder.add_node("detect", detect_sentiment)
builder.add_node("positive", handle_positive)
builder.add_node("negative", handle_negative)
builder.add_node("neutral", handle_neutral)

builder.set_entry_point("detect")

# Conditional logic
def route(state: SentimentState) -> str:
    return state["sentiment"]

builder.add_conditional_edges(
    "detect",
    route,
    {
        "positive": "positive",
        "negative": "negative",
        "neutral": "neutral",
    },
)

# Terminate after handling
builder.add_edge("positive", END)
builder.add_edge("negative", END)
builder.add_edge("neutral", END)

# Compile graph
graph = builder.compile()

# Run it
result = graph.invoke({"input": "I had a terrible day at work."})
print(result["output"])


result: content='\nNegative' additional_kwargs={} response_metadata={'model': 'llama2', 'created_at': '2025-05-06T22:20:04.958740157Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1096641820, 'load_duration': 6850113, 'prompt_eval_count': 54, 'prompt_eval_duration': 228039219, 'eval_count': 5, 'eval_duration': 861005683} id='run-dc0a88b4-eef9-474e-885b-73049a468b2e-0'
I'm sorry to hear that. Want to talk about it?


In [16]:
result = graph.invoke({"input": "I had an amazing day at work."})
print(result["output"])

result: content='\nPositive' additional_kwargs={} response_metadata={'model': 'llama2', 'created_at': '2025-05-06T22:20:35.982336503Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3308394600, 'load_duration': 11149529, 'prompt_eval_count': 55, 'prompt_eval_duration': 2613716451, 'eval_count': 4, 'eval_duration': 682928844} id='run-48cffdbd-78cf-4198-907e-7c0aae491792-0'
That's great to hear! 😊
